In [6]:
#import libraries
from sklearn.datasets import load_boston # dataset
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
import statsmodels.stats.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
import seaborn as sns

%matplotlib inline

In [7]:
# instantiate dataset and create DataFrame 
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
target = pd.DataFrame(boston.target, columns=["MEDV"])

In [8]:
# Basic exploratory analyses 
print("Rows,Columns:",df.shape)
df.describe()

Rows,Columns: (506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [9]:
# split dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size=0.30)


In [10]:
# Fitting using stats model
model_1 = smf.OLS(Y_train,X_train).fit()
prediction_1 = model_1.predict(X_train) # predict y_train values
prediction_2 = model_1.predict(X_test)# predict y_test values

In [11]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     615.2
Date:                Mon, 30 Apr 2018   Prob (F-statistic):          1.12e-227
Time:                        12:04:11   Log-Likelihood:                -1058.9
No. Observations:                 354   AIC:                             2144.
Df Residuals:                     341   BIC:                             2194.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0816      0.051     -1.593      0.112      -0.182       0.019
ZN             0.0439      0.018      2.464      0.014       0.009       0.079
INDUS          0.0136      0.075      0.181      0.856      -0.134       0.161
CHAS           2.4476      1.048      2.335      0.020       0.386       4.510
NOX           -2.2726      4.040     -0.562      0.574     -10.220       5.674
RM             5.9841      0.368     16.254      0.000       5.260       6.708
AGE           -0.0125      0.016     -0.787      0.432      -0.044       0.019
DIS           -0.9827      0.240     -4.086      0.000      -1.456      -0.510
RAD            0.1457      0.079      1.841      0.067      -0.010       0.301
TAX           -0.0091      0.005     -1.981      0.048      -0.018   -6.48e-05
PTRATIO       -0.4227      0.131     -3.226      0.001      -0.680      -0.165
B              0.0152      0.003      4.633      0.000       0.009       0.022
LSTAT         -0.4179      0.059     -7.059      0.000      -0.534      -0.301
==============================================================================
Omnibus:                      170.649   Durbin-Watson:                   1.881
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1525.393
Skew:                           1.802   Prob(JB):                         0.00
Kurtosis:                      12.509   Cond. No.                     8.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Looking at the Summary

In the summary we see some interesting results. The INDUS coefficient (proportion of non-retail business acres per town) is not nearly statistically significant at the 5% or 10% level. The same goes for the AGE and NOX variables.

There seems to be no autocorrelation (as expected as this is not a time-series problem).

Overall, the Adjusted R^2 score is great, but how about RMSE (root mean square error), lets have a loo

In [12]:
rmse_training = sqrt(mean_squared_error(Y_train, prediction_1))
print("RMSE for training data:", rmse_training)
rmse_test = sqrt(mean_squared_error(Y_test,prediction_2))
print("RMSE for test data:", rmse_test)

RMSE for training data: 4.817268734240217
RMSE for test data: 5.165986234287241



Hmm... The RMSE for the test data is quite low and very close to that of the training data. But can we do better?! Lets try by doing a correlation test to identify spurious variables. **A VIF Test.**

In [13]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_test.values, i) for i in range(X_test.shape[1])]
vif["features"] = X_test.columns
vif

,VIF Factor,features
0,1.817784,CRIM
1,3.018425,ZN
2,14.868243,INDUS
3,1.280640,CHAS
4,77.670449,NOX
5,83.613580,RM
6,23.431112,AGE
7,14.110335,DIS
8,15.455333,RAD
9,61.396441,TAX


There is no hard and fast rules on VIF factors, while a score above 10 is undesirable as it suggests very correlation that artificially inflates our R score, a subjective call must be taken depending on the problem. For example, while the TAX variable (full-value property-tax rate per 10,000 dollars) is highly correlated with the rest of the variables, it is too important of a variable to ignore, plus it is very significant at the 5% level.

NOX and INDUS on the other hand, we can try and remove and see if we get a better model. Let's re-create our training sets.

In [14]:

#create second training set
#df = df.drop(["NOX","INDUS"], axis = 1)
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(df, target, test_size=0.30)
print(df.shape)
print(X_test1.shape)

(506, 13)
(152, 13)


In [15]:
# fit second model
model_2 = smf.OLS(Y_train1,X_train1).fit()
prediction_3 = model_2.predict(X_train1)# predict y_test values
prediction_4 = model_2.predict(X_test1)# predict y_test values


In [16]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     624.6
Date:                Mon, 30 Apr 2018   Prob (F-statistic):          9.30e-229
Time:                        12:04:12   Log-Likelihood:                -1064.2
No. Observations:                 354   AIC:                             2154.
Df Residuals:                     341   BIC:                             2205.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0826      0.036     -2.297      0.022      -0.153      -0.012
ZN             0.0450      0.017      2.650      0.008       0.012       0.078
INDUS         -0.0103      0.077     -0.134      0.894      -0.162       0.142
CHAS           3.6785      1.131      3.252      0.001       1.454       5.903
NOX           -0.3803      4.276     -0.089      0.929      -8.792       8.031
RM             6.0116      0.381     15.761      0.000       5.261       6.762
AGE           -0.0093      0.016     -0.569      0.570      -0.041       0.023
DIS           -0.9474      0.225     -4.209      0.000      -1.390      -0.505
RAD            0.2074      0.088      2.368      0.018       0.035       0.380
TAX           -0.0113      0.005     -2.237      0.026      -0.021      -0.001
PTRATIO       -0.3808      0.136     -2.801      0.005      -0.648      -0.113
B              0.0130      0.003      3.847      0.000       0.006       0.020
LSTAT         -0.4880      0.063     -7.791      0.000      -0.611      -0.365
==============================================================================
Omnibus:                      168.255   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1135.309
Skew:                           1.881   Prob(JB):                    2.96e-247
Kurtosis:                      10.926   Cond. No.                     9.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
rmse_training_2 = sqrt(mean_squared_error(Y_train1, prediction_3))
print("RMSE for training data:", rmse_training)
rmse_test_2 = sqrt(mean_squared_error(Y_test1,prediction_4))
print("RMSE for test data:", rmse_test)

RMSE for training data: 4.817268734240217
RMSE for test data: 5.165986234287241


In [18]:
lm = linear_model.LinearRegression()
model_2 = lm.fit(X_train,Y_train)

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [19]:

rms = sqrt(mean_squared_error(Y_test, prediction_2))
print("RMSE_test:",rms)
rms_1 = sqrt(mean_squared_error(Y_train,prediction_1))
print("RMSE_train:",rms_1)

RMSE_test: 5.165986234287241
RMSE_train: 4.817268734240217


In [20]:
print(type(Y_test))

<class 'pandas.core.frame.DataFrame'>


In [21]:
residual = Y_test.values-prediction_2
#print(residual)
for i in X_test:
    plt.scatter(X_test[i],residual)
    plt.xlim(0,)
    plt.xlabel(i)
    plt.ylabel("residuals")
    plt.show()


Exception: Data must be 1-dimensional

## Test for Heteroskedacticity (Breush-Pagan)

Here we are esting for non-constant variance for our error.This violation is most common in cross=sectional data, rarely occuring on purely time-series data.  The example that I had given in class was of comparing state expenditures on Education. So, here MP and UP were large states and chandigah and J&K were small states so we had a non-constant variance of our error terms. One remedy would be to split the states by population and have seperate regressions for states over 10 crore indivuduals and less than 10 crore. 

The breusch pagan test is a hypothesis test, where our Null is that variance is constant and alternative is that variance is non-constant. Therefore our decision hinges on a p-value. 

In [22]:
import statsmodels

name = [ 'p-value']
test = statsmodels.stats.diagnostic.het_breuschpagan(model_1.resid, model_1.model.exog)
lzip(name, test)

[('p-value', 73.91031062416761)]

Here, we cannot reject the null. We are safe from this potential violation.